In [1]:
#Importing and loading libraries
import numpy as np
import pandas as pd
#For PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#For undersampling
from collections import Counter
from imblearn.under_sampling import NearMiss
#Train-test sets splitting
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
#For DecisionTrees
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
#For Logistic Regression classifier
from sklearn.linear_model import LogisticRegression
#For Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB
#For Support Vector Machine
from sklearn import svm
#For Random Forest
from sklearn.ensemble import RandomForestClassifier
#For visualisation
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("C:\\Users\\akram\\Desktop\\Dissertation\\Datasets\\CompleteDataset.csv")
#Removing irrelevant columns
df = df.drop(['Week', 'CalendarDate', 'PlayerName'], axis = 1)
features = df.drop(['InjuredNextTrainingSession'], axis = 1)
target = df['InjuredNextTrainingSession']

In [3]:
#Looking at proportion of '0's and '1's in the target column
imbal_proportion = Counter(target)
print(imbal_proportion)
#Undersampling the training set
undersample = NearMiss(version=3, n_neighbors_ver3=3)
# transform the dataset
features_test, target_test = undersample.fit_resample(features, target)
#Looking at new proportion of '0's and '1's
bal_proportion = Counter(target_test)
print(bal_proportion)

Counter({0: 3805, 1: 38})
Counter({0: 38, 1: 38})


In [4]:
#Function to return accuracy metrics
def accuraciz(y_test, y_pred):
    count = Counter(y_test)
    real_values_zero = count[0]
    real_values_one = count[1]
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    #Transform the data into a numerical vector
    y_test = list(map(int, y_test))
    y_pred = list(map(int, y_pred))
    
    for i in range(len(y_pred)):
        if y_test[i] == 1 and y_pred[i] == 1:
            TP += 1
        if y_pred[i] == 1 and y_test[i] != y_pred[i]:
            FP += 1
        if y_test[i] == y_pred[i] == 0:
            TN += 1
        if y_pred[i] == 0 and y_test[i] != y_pred[i]:
            FN += 1
    
    sensitivity = TP/(TP + FN)
    specificity = TN/(FP + TN)
    FPR = FP / real_values_zero
    FNR = FN / real_values_one
    acc = (TP + TN) / (real_values_one + real_values_zero)
    ba = (sensitivity + specificity) / 2
    f1 = (2*TP) / (2*TP + FP + FN)
    LRplus = sensitivity/(1-specificity)
    LRminus = (1-sensitivity)/specificity
    
    return sensitivity*100, specificity*100, FPR*100, FNR*100, acc*100, ba*100, f1*100, LRplus, LRminus, TP, FP, TN, FN

In [5]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'gini', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 2121 (27.907894736842106%), FP: 1986 (26.13157894736842%)
TN: 1814 (23.86842105263158%), FN: 1679 (22.092105263157894%)
Sensitivity has mean 55.815789473684205 with variance 7.520083102493081
Specificity has mean 47.73684210526315 with variance 15.238227146814413
False Positive Rate has mean 52.26315789473685 with variance 15.23822714681442
False Negative Rate has mean 44.1842105263158 with variance 7.520083102493076
ACC has mean 51.776315789473685 with variance 5.587777008310252
Balanced Accuracy has mean 51.776315789473685 with variance 5.5877770083102485
F1-Score has mean 53.64140863778516 with variance 4.708559212096166
Likelihood ratio positive has mean 1.0737874991963365 with variance 0.008821821706490991
Likelihood ratio negative has mean 0.9320085753599686 with variance 0.009741726014405913


In [6]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myDT = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_myDT = cross_val_predict(myDT, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myDT)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 2078 (27.342105263157894%), FP: 2027 (26.67105263157895%)
TN: 1773 (23.32894736842105%), FN: 1722 (22.657894736842106%)
Sensitivity has mean 54.684210526315795 with variance 11.991689750692535
Specificity has mean 46.65789473684211 with variance 28.234764542936304
False Positive Rate has mean 53.34210526315789 with variance 28.234764542936304
False Negative Rate has mean 45.31578947368422 with variance 11.991689750692524
ACC has mean 50.67105263157895 with variance 8.258137119113572
Balanced Accuracy has mean 50.67105263157895 with variance 8.258137119113574
F1-Score has mean 52.559680070919995 with variance 6.574402810942533
Likelihood ratio positive has mean 1.0342652881379997 with variance 0.01265937569079536
Likelihood ratio negative has mean 0.9826268571918054 with variance 0.015758305304392716


In [7]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Decision Tree classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myCLR = LogisticRegression(solver='lbfgs', max_iter=1000)

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_myCLR = cross_val_predict(myCLR, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myCLR)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1
    
TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

...

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/m

TP: 2100 (27.63157894736842%), FP: 1900 (25.0%)
TN: 1900 (25.0%), FN: 1700 (22.36842105263158%)
Sensitivity has mean 55.26315789473686 with variance 5.048709793414476e-29
Specificity has mean 50.0 with variance 0.0
False Positive Rate has mean 50.0 with variance 0.0
False Negative Rate has mean 44.73684210526318 with variance 4.543838814073028e-28
ACC has mean 52.631578947368425 with variance 5.048709793414476e-29
Balanced Accuracy has mean 52.631578947368425 with variance 5.048709793414476e-29
F1-Score has mean 53.84615384615387 with variance 4.543838814073028e-28
Likelihood ratio positive has mean 1.105263157894737 with variance 0.0
Likelihood ratio negative has mean 0.894736842105263 with variance 0.0


C:\Users\akram\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Naive-Bayes classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myNB = GaussianNB()

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_myNB = cross_val_predict(myNB, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myNB)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 2900 (38.1578947368421%), FP: 2300 (30.263157894736842%)
TN: 1500 (19.736842105263158%), FN: 900 (11.842105263157896%)
Sensitivity has mean 76.3157894736842 with variance 2.0194839173657902e-28
Specificity has mean 39.4736842105263 with variance 2.0194839173657902e-28
False Positive Rate has mean 60.526315789473664 with variance 4.543838814073028e-28
False Negative Rate has mean 23.684210526315788 with variance 0.0
ACC has mean 57.89473684210527 with variance 0.0
Balanced Accuracy has mean 57.89473684210527 with variance 0.0
F1-Score has mean 64.44444444444444 with variance 0.0
Likelihood ratio positive has mean 1.2608695652173911 with variance 4.930380657631324e-32
Likelihood ratio negative has mean 0.5999999999999998 with variance 1.232595164407831e-32


In [9]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
mySVC = svm.SVC()

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_mySVC = cross_val_predict(mySVC, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_mySVC)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 3300 (43.421052631578945%), FP: 2400 (31.57894736842105%)
TN: 1400 (18.42105263157895%), FN: 500 (6.578947368421052%)
Sensitivity has mean 86.84210526315792 with variance 2.0194839173657902e-28
Specificity has mean 36.8421052631579 with variance 5.048709793414476e-29
False Positive Rate has mean 63.15789473684211 with variance 5.048709793414476e-29
False Negative Rate has mean 13.157894736842106 with variance 3.1554436208840472e-30
ACC has mean 61.84210526315789 with variance 5.048709793414476e-29
Balanced Accuracy has mean 61.84210526315789 with variance 5.048709793414476e-29
F1-Score has mean 69.47368421052634 with variance 8.077935669463161e-28
Likelihood ratio positive has mean 1.3750000000000002 with variance 0.0
Likelihood ratio negative has mean 0.357142857142857 with variance 3.0814879110195774e-33


In [10]:
i = 1
sensitivities = []
specificities = []
FPRs = []
FNRs = []
ACCs = []
BAs = []
F1s = []
LRpluses = []
LRminuses = []
TPs = []
FPs = []
TNs = []
FNs = []

#Building the Support Vector classifier
#Recursive to obtain mean and variance of results
#10-fold cross validation
myRF = RandomForestClassifier(criterion = 'entropy', max_depth = 10)

while i < 101:
    features_kfold, target_kfold = undersample.fit_resample(features, target)
    y_pred_myRF = cross_val_predict(myRF, features_kfold, target_kfold, cv=10)

    performance = accuraciz(target_kfold, y_pred_myRF)
    sensitivities.append(performance[0])
    specificities.append(performance[1])
    FPRs.append(performance[2])
    FNRs.append(performance[3])
    ACCs.append(performance[4])
    BAs.append(performance[5])
    F1s.append(performance[6])
    LRpluses.append(performance[7])
    LRminuses.append(performance[8])
    TPs.append(performance[9])
    FPs.append(performance[10])
    TNs.append(performance[11])
    FNs.append(performance[12])
    
    i += 1

TPz = np.sum(TPs)
FPz = np.sum(FPs)
TNz = np.sum(TNs)
FNz = np.sum(FNs)
totz = TPz + FPz + TNz + FNz

print("TP: "+str(TPz)+" ("+str(TPz*100/totz)+"%), FP: "+str(FPz)+" ("+str(FPz*100/totz)+"%)")
print("TN: "+str(TNz)+" ("+str(TNz*100/totz)+"%), FN: "+str(FNz)+" ("+str(FNz*100/totz)+"%)")
print("Sensitivity has mean "+str(np.mean(sensitivities))+" with variance "+str(np.var(sensitivities)))
print("Specificity has mean "+str(np.mean(specificities))+" with variance "+str(np.var(specificities)))
print("False Positive Rate has mean "+str(np.mean(FPRs))+" with variance "+str(np.var(FPRs)))
print("False Negative Rate has mean "+str(np.mean(FNRs))+" with variance "+str(np.var(FNRs)))
print("ACC has mean "+str(np.mean(ACCs))+" with variance "+str(np.var(ACCs)))
print("Balanced Accuracy has mean "+str(np.mean(BAs))+" with variance "+str(np.var(BAs)))
print("F1-Score has mean "+str(np.mean(F1s))+" with variance "+str(np.var(F1s)))
print("Likelihood ratio positive has mean "+str(np.mean(LRpluses))+" with variance "+str(np.var(LRpluses)))
print("Likelihood ratio negative has mean "+str(np.mean(LRminuses))+" with variance "+str(np.var(LRminuses)))

TP: 2201 (28.960526315789473%), FP: 1704 (22.42105263157895%)
TN: 2096 (27.57894736842105%), FN: 1599 (21.039473684210527%)
Sensitivity has mean 57.92105263157895 with variance 8.37880886426592
Specificity has mean 55.15789473684211 with variance 13.008310249307492
False Positive Rate has mean 44.8421052631579 with variance 13.008310249307485
False Negative Rate has mean 42.07894736842106 with variance 8.37880886426593
ACC has mean 56.53947368421053 with variance 5.382790858725761
Balanced Accuracy has mean 56.53947368421053 with variance 5.382790858725762
F1-Score has mean 57.1209590825704 with variance 5.143344776759513
Likelihood ratio positive has mean 1.3000939235343258 with variance 0.015418379600054403
Likelihood ratio negative has mean 0.7662535141232137 with variance 0.005423605537824306
